In [1]:
"""
等间距（默认 15 m）采样多边形，用 streetview.find_panorama()
把 Google Street View 全景点补齐，输出 id/lat/lon/date 到 CSV
"""
import math, csv, time
from pathlib import Path

import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
from streetlevel import streetview

# ---------- 参数 ----------
shp_path        = r"test/test.shp"          # ← 你的多边形 Shapefile
step_m          = 1                        # 采样网格步长
search_radius_m = 15                        # find_panorama 搜索半径
max_retry       = 3
out_csv         = Path(r"D:\tree4heatland\model\streetview-dl-main\pano_metadata2.csv")

# ---------- 日期格式化 ----------
def capturedate_to_str(cd):
    """CaptureDate → 'YYYY-MM' 或 'YYYY-MM-DD' 字符串"""
    if not cd:
        return ""
    y, m = cd.year, cd.month
    d = getattr(cd, "day", None)
    return f"{y}-{m:02d}-{d:02d}" if d else f"{y}-{m:02d}"

# ---------- 1. 读多边形并转米制 ----------
poly = gpd.read_file(shp_path).unary_union
poly_m = gpd.GeoSeries([poly], crs=4326).to_crs(3857).iloc[0]
minx, miny, maxx, maxy = poly_m.bounds

# ---------- 2. 生成点阵 ----------
xs = list(range(int(minx), int(maxx) + 1, step_m))
ys = list(range(int(miny), int(maxy) + 1, step_m))

# ---------- 3. find_panorama 扫描 ----------
meta = {}           # pano_id -> dict
total_pts = len(xs) * len(ys)
pbar = tqdm(total=total_pts, desc="grid")

for x in xs:
    for y in ys:
        pbar.update()
        pt = Point(x, y)
        if not poly_m.contains(pt):
            continue

        # 投回 WGS84
        lon, lat = (
            gpd.GeoSeries([pt], crs=3857).to_crs(4326).iloc[0].coords[0]
        )

        pano = None
        for _ in range(max_retry):
            try:
                pano = streetview.find_panorama(
                    lat=lat,
                    lon=lon,
                    radius=search_radius_m,
                    session=None,             # 默认 requests
                    search_third_party=False  # 如需第三方设 True
                )
                break
            except Exception:
                time.sleep(0.3)

        if pano and pano.id not in meta:
            meta[pano.id] = {
                "lat": pano.lat,
                "lon": pano.lon,
                "date": capturedate_to_str(pano.date)
            }

pbar.close()
print(f"✅ 找到 {len(meta)} 个 pano")

# ---------- 4. 写 CSV ----------
out_csv.parent.mkdir(parents=True, exist_ok=True)
with out_csv.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=["id", "lat", "lon", "date"])
    w.writeheader()
    for pid, row in meta.items():
        w.writerow({"id": pid, **row})

print(f"[✓] CSV 保存至 {out_csv}")


C:\Users\HIT\AppData\Local\Temp\ipykernel_76708\2670195135.py:30: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  poly = gpd.read_file(shp_path).unary_union
grid: 100%|██████████| 112000/112000 [8:04:41<00:00,  3.85it/s]   

✅ 找到 153 个 pano
[✓] CSV 保存至 D:\tree4heatland\model\streetview-dl-main\pano_metadata2.csv
